In [46]:
!pip install openai
!pip install pymupdf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/16.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.0 MB ? eta -:--:--
    --------------------------------------- 0.3/16.0 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.0 MB 1.4 MB/s eta 0:00:11
   -- ------------------------------------- 1.0/16.0 MB 1.5 MB/s eta 0:00:11
   --- ------------------------------------ 1.6/16.0 MB 1.7 MB/s eta 0:00:09
   ----- ---------------------------------- 2.1/16.0 MB 1.8 MB/s eta 0:00:08
   ------ --------------------------------- 2.6/16.0 MB 2.0 MB/s eta 0:00:07
   ------- -------------------------------- 3.1/16.0 MB 2.1 MB/s eta 0:00:07
   --------- ------------------------------ 3.9/16.0 MB 2.3 MB/s eta 0:00:06
   ----------- ---------------------------- 4.5/16.0 MB 2.4 MB/s eta 0:00:05
   ------------- -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
import nest_asyncio
import os

# Apply nest_asyncio to handle nested event loops (useful for Jupyter notebooks)
nest_asyncio.apply()

# Ensure the OpenAI API key is set as an environment variable
assert "OPENAI_API_KEY" in os.environ, "Please set the OPENAI_API_KEY environment variable."

# Import the OpenAI and embedding classes from Llama-Index
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# Initialize the language model (LLM) using gpt-4o-mini and embedding model
llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
embed_model = OpenAIEmbedding()

# Set the LLM and embedding model globally for usage
Settings.llm = llm
Settings.embed_model = embed_model

In [51]:
import fitz  # PyMuPDF
from llama_index.core import Document

# Load the PDF and split by pages
pdf_path = "acura_mdx_manual.pdf"
pdf_document = fitz.open(pdf_path)

# Create a list of Document objects with page-level metadata
acura_docs = []
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]
    page_text = page.get_text("text")
    document = Document(text=page_text, metadata={"page": page_num + 1})
    acura_docs.append(document)

In [48]:
# from llama_index.core import SimpleDirectoryReader

# # Load the Acura MDX manual
# acura_docs = SimpleDirectoryReader(input_files=["acura_mdx_manual.pdf"]).load_data()

In [53]:
from llama_index.core import VectorStoreIndex

# Create vector store index from the Acura MDX manual
acura_index = VectorStoreIndex.from_documents(acura_docs)

# Create a query engine for the Acura manual
acura_query_engine = acura_index.as_query_engine(similarity_top_k=3)

In [61]:
# Query the Acura MDX manual for tire pressure check recommendations
query = "How often should tire pressure be checked, especially during cold weather?"
response = acura_query_engine.query(query)

# # Print the response attributes to check for 'source_documents'
# print("Response structure:")
# print(response.__dict__)  # Check all attributes of the response

# Display the response and relevant excerpts
from IPython.display import display, HTML

# Display main response
display(HTML(f'<p style="font-size:20px; color: darkblue;"><strong>Response:</strong> {response.response}</p>'))

# Display excerpts from source_nodes
if hasattr(response, 'source_nodes') and response.source_nodes:
    for i, node in enumerate(response.source_nodes):
        page_info = f"Page {node.node.metadata.get('page', 'N/A')}" if node.node.metadata else "Unknown page"
        excerpt = node.node.text[:500]  # Limit excerpt length to 500 characters
        display(HTML(f'<p style="font-size:16px; color: darkgreen;"><strong>Excerpt from {page_info}:</strong><br>{excerpt}...</p>'))
else:
    display(HTML("<p style='font-size:16px; color: red;'>No excerpts found in the response.</p>"))

In [65]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import FunctionCallingAgentWorker

# Define the query engine tool for Acura manual
query_engine_tools = [
    QueryEngineTool(
        query_engine=acura_query_engine,
        metadata=ToolMetadata(
            name="acura_manual",
            description="Provides information from the Acura MDX 2022 owner's manual",
        ),
    )
]

# Create a function-calling agent worker
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)

# Convert the agent worker to an agent
agent = agent_worker.as_agent()

# Use the agent to ask a question about the Acura manual
response = agent.chat("How often should tire pressure be checked, especially during cold weather?")
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

Added user message to memory: How often should tire pressure be checked, especially during cold weather?
=== Calling Function ===
Calling function: acura_manual with args: {"input": "tire pressure check frequency cold weather"}
=== Function Output ===
Tire pressure should be checked monthly when the tires are cold. This means the vehicle should have been parked for at least three hours or driven less than 1 mile (1.6 km) before checking the pressure. Regular checks are especially important in cold weather, as temperatures can cause tire pressure to drop.
=== LLM Response ===
Tire pressure should be checked monthly, especially during cold weather. It's best to check the pressure when the tires are cold, meaning the vehicle should have been parked for at least three hours or driven less than 1 mile (1.6 km) before checking. Cold temperatures can cause tire pressure to drop, making regular checks crucial.


# Agentic Architecture Overview

The setup using `FunctionCallingAgentWorker` with Llama Index can be considered an example of agentic architecture. Here's a breakdown of why this approach qualifies:

1. **Agents and Autonomy**  
   - The `FunctionCallingAgentWorker` creates an agent that autonomously decides which tools (query engines) to utilize based on the user's query.
   - This makes it an "agent" because it can perform actions independently to resolve queries. For instance, the agent autonomously decides which section of the manual to query to answer a question.

2. **Tool Integration**  
   - The agent is integrated with "tools" (`QueryEngineTool`), which provide specific capabilities—in this case, querying the Acura manual data.
   - This tool integration is central to agentic architecture as it allows the agent to perform specialized tasks using pre-defined functionalities.

3. **Reasoning and Function Calling**  
   - The `FunctionCallingAgentWorker` allows the agent to reason and call specific functions as needed based on the user's prompt.
   - This setup enables the agent to make decisions and take actions, such as querying the Acura manual for specific information like resetting the oil change light.

## Differences from a Basic Query System
- A basic query system only returns search results without processing or reasoning, while the agentic approach "thinks through" the required steps.
- Agentic architecture enables multiple decision-making steps and tool usage, adding sophistication beyond simple query-response mechanisms.

## Benefits
- **Modularity**: Additional tools can be added to the agent, enabling it to autonomously decide when to use each one.
- **Scalability**: The agent can scale to handle complex, multi-step queries and interactions, making it more versatile than a basic query engine.

In summary, this setup leverages principles of agentic architecture, enabling it to dynamically and autonomously interact with users' queries. This is beneficial for scenarios that require more than simple responses, making it capable of sophisticated, contextualized interactions.

In [42]:
# Query the Acura MDX manual for tire pressure check recommendations
query = "How often should tire pressure be checked, especially during cold weather?"
response = acura_query_engine.query(query)

# Display the response and relevant excerpts
from IPython.display import display, HTML

# Display main response
display(HTML(f'<p style="font-size:20px; color: darkblue;"><strong>Response:</strong> {response.response}</p>'))

# Display excerpts if available
if hasattr(response, 'source_documents') and response.source_documents:
    for i, doc in enumerate(response.source_documents):
        page_info = f"Page {doc.metadata.get('page', 'N/A')}" if doc.metadata else "Unknown page"
        excerpt = doc.text[:500]  # Limit excerpt length to 500 characters
        display(HTML(f'<p style="font-size:16px; color: darkgreen;"><strong>Excerpt from {page_info}:</strong><br>{excerpt}...</p>'))
else:
    display(HTML("<p style='font-size:16px; color: red;'>No excerpts found in the response.</p>"))